In [1]:
import pandas as pd

detalle = pd.read_csv("ListaCobroDetalle2024.csv")  # Archivo grande
respuestas = pd.read_csv("CatRespuestaBancos.csv")
emisoras = pd.read_csv("CatEmisora.csv")
relacion = pd.read_csv("ListaCobroEmisora.csv")
print(respuestas.columns.tolist())
print(detalle.columns.tolist())

['IdRespuestaBanco', 'Descripcion']
['idListaCobro', 'idCredito', 'consecutivoCobro', 'idBanco', 'montoExigible', 'montoCobrar', 'montoCobrado', 'fechaCobroBanco', 'idRespuestaBanco']


In [2]:
# 🔤 Convertir columna clave a string para evitar conflictos
detalle['idRespuestaBanco'] = detalle['idRespuestaBanco'].astype(str)

# 🔗 Establecer índice en respuestas y asegurar tipo string
respuestas = respuestas.set_index('IdRespuestaBanco')
respuestas.index = respuestas.index.astype(str)

# ⚡ Join rápido con columna 'Descripcion'
detalle = detalle.join(respuestas[['Descripcion']], on='idRespuestaBanco')

# 🔍 Crear columna en minúsculas para analizar tipo de respuesta
detalle['descripcion_lower'] = detalle['Descripcion'].fillna('').str.lower()

# 🚨 Detectar si fue un intento bloqueado
detalle['es_bloqueo'] = detalle['descripcion_lower'].str.contains(
    'bloquead|cerrad|cancelad|inexistente|rechazad|no existe|inactiva|suspendid'
)

# ⚠️ Eliminar 'idEmisora' antes del merge si ya existe
if 'idEmisora' in detalle.columns:
    detalle = detalle.drop(columns=['idEmisora'])

# 🔗 Merge con relación y catálogo de emisoras
detalle = detalle.merge(relacion, on='idListaCobro', how='left')
detalle = detalle.merge(emisoras, on='idEmisora', how='left')

# 🧠 Etiqueta simplificada de tipo de estrategia
detalle['estrategia_simplificada'] = detalle['TipoEnvio'].fillna('SIN_DATO')

In [3]:
from datetime import datetime

# ✅ Asegurar tipos correctos para columnas clave
detalle['fechaCobroBanco'] = pd.to_datetime(detalle['fechaCobroBanco'], errors='coerce')
detalle['montoCobrado'] = pd.to_numeric(detalle['montoCobrado'], errors='coerce')
detalle['montoCobrar'] = pd.to_numeric(detalle['montoCobrar'], errors='coerce')

if 'montoExigible' in detalle.columns:
    detalle['montoExigible'] = pd.to_numeric(detalle['montoExigible'], errors='coerce')

# 🧠 Función auxiliar para evaluar éxito reciente
hoy = pd.Timestamp.now()

def hubo_exito_reciente(df):
    recientes = df[df['fechaCobroBanco'] > hoy - pd.Timedelta(days=365)]
    return (recientes['montoCobrado'] > 0).any()

# 📊 Agrupación por cliente con cálculo de métricas
resumen = detalle.groupby('idCredito').apply(lambda g: pd.Series({
    'intentos_totales': g.shape[0],
    'intentos_exitosos': (g['montoCobrado'] > 0).sum(),
    'intentos_bloqueados': g['es_bloqueo'].sum(),
    'ultimo_exito': g[g['montoCobrado'] > 0]['fechaCobroBanco'].max(),
    'primer_intento': g['fechaCobroBanco'].min(),
    'monto_total': g['montoCobrar'].sum(),
    'exito_reciente': hubo_exito_reciente(g),
    'num_estrategias_diferentes': g['estrategia_simplificada'].nunique()
})).reset_index()

# 🧮 Derivar antigüedad y tiempo sin éxito
resumen['antiguedad_anios'] = (hoy - resumen['primer_intento']).dt.days / 365
resumen['sin_exito_desde'] = (hoy - resumen['ultimo_exito']).dt.days / 365

# 🧠 Aplicación de reglas para clasificación estratégica
resumen['clasificacion'] = 'sin_clasificar'

resumen.loc[resumen['intentos_bloqueados'] > 0, 'clasificacion'] = 'interjudicial'

resumen.loc[
    (resumen['clasificacion'] == 'sin_clasificar') &
    (
        (resumen['intentos_totales'] >= 10) |
        (resumen['antiguedad_anios'] >= 4) |
        (resumen['monto_total'] < 50)
    ), 'clasificacion'
] = 'dar_por_perdido'

resumen.loc[
    (resumen['clasificacion'] == 'sin_clasificar') &
    (resumen['exito_reciente']), 'clasificacion'
] = 'mantener_estrategia'

resumen.loc[
    (resumen['clasificacion'] == 'sin_clasificar') &
    (resumen['num_estrategias_diferentes'] == 1), 'clasificacion'
] = 'probar_otra_estrategia'

resumen.loc[resumen['clasificacion'] == 'sin_clasificar', 'clasificacion'] = 'intentar_por_bbva'


<ipython-input-3-408a27655022>:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resumen = detalle.groupby('idCredito').apply(lambda g: pd.Series({


In [4]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

X1 = resumen[[
    'intentos_totales', 'intentos_exitosos', 'intentos_bloqueados',
    'monto_total', 'antiguedad_anios', 'sin_exito_desde', 'num_estrategias_diferentes'
]].fillna(0)

y1 = resumen['clasificacion']
label_encoder1 = LabelEncoder()
y1_encoded = label_encoder1.fit_transform(y1)

scaler1 = StandardScaler()
X1_scaled = scaler1.fit_transform(X1)

X1_train, X1_test, y1_train, y1_test = train_test_split(X1_scaled, y1_encoded, test_size=0.2, random_state=42)

y1_train_cat = to_categorical(y1_train)
y1_test_cat = to_categorical(y1_test)

model1 = Sequential([
    Dense(64, activation='relu', input_shape=(X1_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(y1_train_cat.shape[1], activation='softmax')
])

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model1.fit(X1_train, y1_train_cat, epochs=20, batch_size=512, validation_split=0.2)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.4747 - loss: 1.2251 - val_accuracy: 0.7659 - val_loss: 0.7393
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7703 - loss: 0.6755 - val_accuracy: 0.7729 - val_loss: 0.5696
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7858 - loss: 0.5307 - val_accuracy: 0.8163 - val_loss: 0.4704
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8284 - loss: 0.4468 - val_accuracy: 0.8307 - val_loss: 0.4009
Epoch 5/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8361 - loss: 0.3927 - val_accuracy: 0.8422 - val_loss: 0.3518
Epoch 6/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8521 - loss: 0.3429 - val_accuracy: 0.8534 - val_loss: 0.3139
Epoch 7/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8572 - loss: 0.3063 - val_accuracy: 0.8635 - val_loss: 0.2814
Epoch 8/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8728 - loss: 0.2820 - val_accuracy: 0.8730 - val_loss: 0.2581
Epo

In [5]:
activos = resumen[resumen['clasificacion'].isin(['mantener_estrategia', 'probar_otra_estrategia', 'intentar_por_bbva'])]['idCredito']
detalle_activo = detalle[detalle['idCredito'].isin(activos)]
print(detalle_activo.columns.tolist())

['idListaCobro', 'idCredito', 'consecutivoCobro', 'idBanco', 'montoExigible', 'montoCobrar', 'montoCobrado', 'fechaCobroBanco', 'idRespuestaBanco', 'Descripcion', 'descripcion_lower', 'es_bloqueo', 'idEmisora', 'Nombre', 'IdBanco', 'Emisora', 'TipoEnvio', 'estrategia_simplificada']


In [6]:
detalle_activo['montoCobrar'] = pd.to_numeric(detalle_activo['montoCobrar'], errors='coerce')
detalle_activo['montoCobrado'] = pd.to_numeric(detalle_activo['montoCobrado'], errors='coerce')

# Función para asignar costo real por banco y estrategia
def costo_real(row):
    banco = row['Nombre'].upper() if pd.notnull(row['Nombre']) else ''
    tipo = row['TipoEnvio'].upper() if pd.notnull(row['TipoEnvio']) else ''
    estrategia = str(row['Emisora']).strip()

    # BANAMEX - cobra por registro enviado
    if "BANAMEX" in banco:
        return 1.75

    # BBVA - cobra por registro exitoso
    if "BBVA" in banco:
        if "PARCIAL" in tipo or "06111" in estrategia:
            return 1.60
        elif "MATUTINO" in tipo or "06114" in estrategia:
            return 1.91
        elif "EN LINEA" in tipo or "07455" in estrategia:
            return 8.00
        elif "INTERBANCAR" in tipo:
            return 4.00
        else:
            return 0.80  # TRADICIONAL

    # SANTANDER - cobra al final del monitoreo
    if "SANTANDER" in banco:
        if "INTERBANCAR" in tipo:
            return 3.18 if row['montoCobrado'] > 0 else 2.58
        elif "H2H" in tipo:
            return 2.20 if row['montoCobrado'] > 0 else 1.90
        else:
            return 2.82 if row['montoCobrado'] > 0 else 2.37

    # BANORTE - cobra por registro enviado, o especializado si aplica
    if "BANORTE" in banco:
        if "ESPECIALIZADO" in tipo.upper():
            return 20.00
        elif "INTERBANCAR" in tipo.upper():
            return 4.50
        else:
            return 2.50

    # Default general
    return 3.00

# Aplicar la función
detalle_activo['Costo'] = detalle_activo.apply(costo_real, axis=1)

# Calcular ganancia y ahorro reales
detalle_activo['ganancia'] = (detalle_activo['montoCobrado'] - detalle_activo['Costo']).fillna(0)

costo_maximo = detalle_activo.groupby('idCredito')['Costo'].transform('max')
detalle_activo['ahorro'] = (costo_maximo - detalle_activo['Costo']).fillna(0)

<ipython-input-6-85b2f44b3fb9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detalle_activo['montoCobrar'] = pd.to_numeric(detalle_activo['montoCobrar'], errors='coerce')
<ipython-input-6-85b2f44b3fb9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detalle_activo['montoCobrado'] = pd.to_numeric(detalle_activo['montoCobrado'], errors='coerce')
<ipython-input-6-85b2f44b3fb9>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [7]:
# Etiqueta de clasificación
detalle_activo['label_clase'] = detalle_activo['idEmisora'].astype(int)

from sklearn.preprocessing import OneHotEncoder

# Features
features2 = [
    'montoCobrar', 'montoCobrado', 'montoExigible', 'idBanco', 'idCredito',
    'idRespuestaBanco', 'fechaCobroBanco'
]
detalle_activo['fechaCobroBanco'] = pd.to_datetime(detalle_activo['fechaCobroBanco'], errors='coerce')
detalle_activo['diaCobro'] = detalle_activo['fechaCobroBanco'].dt.dayofweek
detalle_activo['horaCobro'] = detalle_activo['fechaCobroBanco'].dt.hour

X2 = detalle_activo[['montoCobrar', 'montoCobrado', 'montoExigible', 'idBanco', 'idCredito',
                     'idRespuestaBanco', 'diaCobro', 'horaCobro']].fillna(0)

y2_reg = detalle_activo[['ahorro', 'ganancia']].astype('float32').fillna(0)

# Encoding
label_encoder2 = LabelEncoder()
y2_class = detalle_activo['label_clase']  # aseguramos que sea idEmisora
y2_class_encoded = label_encoder2.fit_transform(y2_class)

scaler2 = StandardScaler()
X2_scaled = scaler2.fit_transform(X2)

X2_train, X2_test, y2r_train, y2r_test = train_test_split(X2_scaled, y2_reg, test_size=0.2, random_state=42)
X2c_train, X2c_test, y2c_train, y2c_test = train_test_split(X2_scaled, to_categorical(y2_class_encoded), test_size=0.2, random_state=42)

<ipython-input-7-15ce6b3d5ed7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detalle_activo['label_clase'] = detalle_activo['idEmisora'].astype(int)
<ipython-input-7-15ce6b3d5ed7>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detalle_activo['fechaCobroBanco'] = pd.to_datetime(detalle_activo['fechaCobroBanco'], errors='coerce')
<ipython-input-7-15ce6b3d5ed7>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [8]:
import numpy as np

print("NaNs en X2_train:", np.isnan(X2_train).sum())
print("Inf en X2_train:", np.isinf(X2_train).sum())
print("NaNs en y2r_train:", np.isnan(y2r_train).sum())
print("Inf en y2r_train:", np.isinf(y2r_train).sum())

import numpy as np

# Filas válidas (sin NaN ni Inf)
valid_rows = ~np.isnan(X2_train).any(axis=1) & ~np.isinf(X2_train).any(axis=1)

# Aplicar filtrado a ambos
X2_train_clean = X2_train[valid_rows]
y2r_train_clean = y2r_train[valid_rows]

NaNs en X2_train: 4016
Inf en X2_train: 0
NaNs en y2r_train: ahorro      0
ganancia    0
dtype: int64
Inf en y2r_train: ahorro      0
ganancia    0
dtype: int64


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
import numpy as np

# --- REGRESIÓN (ya estaba bien con datos limpios) ---
model2_reg = Sequential([
    Input(shape=(X2_train_clean.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(2)
])
model2_reg.compile(optimizer='adam', loss='mse', metrics=['mae'])
model2_reg.fit(X2_train_clean, y2r_train_clean, epochs=20, batch_size=512, validation_split=0.2)

# --- CLASIFICACIÓN ---
# Limpieza para clasificación
valid_cls = ~np.isnan(X2_scaled).any(axis=1) & ~np.isinf(X2_scaled).any(axis=1)
X2c_clean = X2_scaled[valid_cls]

# Recalcular etiquetas codificadas SOLO para estas filas
y2_class_clean = y2_class[valid_cls]
y2_class_encoded_clean = label_encoder2.transform(y2_class_clean)
y2c_clean = to_categorical(y2_class_encoded_clean)

# Separar entrenamiento y test para clasificación
X2c_train, X2c_test, y2c_train, y2c_test = train_test_split(X2c_clean, y2c_clean, test_size=0.2, random_state=42)

# Modelo de clasificación corregido
model2_class = Sequential([
    Input(shape=(X2c_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(y2c_train.shape[1], activation='softmax')
])
model2_class.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2_class.fit(X2c_train, y2c_train, epochs=20, batch_size=512, validation_split=0.2)

Epoch 1/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 176172.2344 - mae: 97.1255 - val_loss: 185886.7656 - val_mae: 97.7775
Epoch 2/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 168584.4844 - mae: 94.0251 - val_loss: 181715.0000 - val_mae: 98.1658
Epoch 3/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 161512.4531 - mae: 94.2764 - val_loss: 168191.2500 - val_mae: 96.6831
Epoch 4/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 159938.1562 - mae: 93.3678 - val_loss: 141348.2031 - val_mae: 91.7051
Epoch 5/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 130675.7891 - mae: 88.6212 - val_loss: 104047.8359 - val_mae: 83.6510
Epoch 6/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 88769.6094 - mae: 80.2846 - val_loss: 68990.2969 - val_mae: 73.5322
Epoch 7/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 60780.1562 - mae: 69.9371 - val_loss: 43445.5859 - val_mae: 62.3300
Epoch 8/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 37215.4766 - mae: 59.5431 - val_loss: 26413.7598 -

In [10]:
import numpy as np
import re
import pandas as pd

# 🔮 Predicciones
assert X2_scaled.shape[0] == detalle_activo.shape[0], "El número de filas no coincide entre X2_scaled y detalle_activo"
pred_reg = model2_reg.predict(X2_scaled)
pred_class_probs = model2_class.predict(X2_scaled)
pred_class = np.argmax(pred_class_probs, axis=1)

# 📌 Asignación segura
detalle_activo.loc[:, 'pred_ahorro'] = pred_reg[:, 0]
detalle_activo.loc[:, 'pred_ganancia'] = pred_reg[:, 1]
detalle_activo.loc[:, 'mejor_emisora_clase'] = label_encoder2.inverse_transform(pred_class)

detalle_activo['idEmisora_predicha'] = detalle_activo['mejor_emisora_clase'].astype(int)

# 📤 Exportar plan general
detalle_activo[['idCredito', 'consecutivoCobro', 'pred_ahorro', 'pred_ganancia', 'mejor_emisora_clase']] \
    .to_csv("plan_accion_cobranza.csv", index=False)

# 📊 Resumen por emisora sugerida
resumen_emisoras = detalle_activo.groupby('idEmisora_predicha').agg(
    total_creditos=('idCredito', 'nunique'),
    total_ahorro=('pred_ahorro', 'sum'),
    total_ganancia=('pred_ganancia', 'sum'),
    promedio_ahorro=('pred_ahorro', 'mean'),
    promedio_ganancia=('pred_ganancia', 'mean')
).reset_index().sort_values(by='total_ganancia', ascending=False)

resumen_emisoras.to_csv("resumen_por_emisora.csv", index=False)

# 📥 Comparativa real vs predicha
relacion = pd.read_csv("/content/ListaCobroEmisora.csv")  # Ajusta si ya estaba cargado

# Asegura que las columnas estén nombradas correctamente
relacion = relacion.rename(columns={'idEmisora': 'idEmisora_real'})

# Unir con emisora real
detalle_comparado = detalle_activo.merge(
    relacion[['idListaCobro', 'idEmisora_real']],
    on='idListaCobro',
    how='left'
)

# Extraer columnas clave
comparacion_final = detalle_comparado[[
    'idCredito', 'consecutivoCobro', 'idEmisora_real', 'idEmisora_predicha', 'pred_ahorro', 'pred_ganancia'
]]

# Exportar archivo único comparativo
comparacion_final.to_csv("comparacion_emisora_real_vs_predicha.csv", index=False)

print("\n✅ Archivo 'comparacion_emisora_real_vs_predicha.csv' generado correctamente.")

1548/1548 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
1548/1548 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


<ipython-input-10-ca347add1bb8>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detalle_activo.loc[:, 'pred_ahorro'] = pred_reg[:, 0]
<ipython-input-10-ca347add1bb8>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detalle_activo.loc[:, 'pred_ganancia'] = pred_reg[:, 1]
<ipython-input-10-ca347add1bb8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.


✅ Archivo 'comparacion_emisora_real_vs_predicha.csv' generado correctamente.


In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
import numpy as np
import joblib

# -----------------------------
# PARTE 1: CARGA DE DATOS
# -----------------------------
detalle = pd.read_csv("ListaCobroDetalle2024.csv")
respuestas = pd.read_csv("CatRespuestaBancos.csv")
emisoras = pd.read_csv("CatEmisora.csv")
relacion = pd.read_csv("ListaCobroEmisora.csv")
observaciones = pd.read_excel("Tabla_con_Observaciones.xlsx")

# Preprocesamiento
detalle['idRespuestaBanco'] = detalle['idRespuestaBanco'].astype(str)
respuestas = respuestas.set_index('IdRespuestaBanco')
respuestas.index = respuestas.index.astype(str)
detalle = detalle.join(respuestas[['Descripcion']], on='idRespuestaBanco')
detalle['descripcion_lower'] = detalle['Descripcion'].fillna('').str.lower()
detalle['es_bloqueo'] = detalle['descripcion_lower'].str.contains('bloquead|cerrad|cancelad|inexistente|rechazad|no existe|inactiva|suspendid')

if 'idEmisora' in detalle.columns:
    detalle = detalle.drop(columns=['idEmisora'])
detalle = detalle.merge(relacion, on='idListaCobro', how='left')
detalle = detalle.merge(emisoras, on='idEmisora', how='left')

detalle['estrategia_simplificada'] = detalle['TipoEnvio'].fillna('SIN_DATO')
detalle['fechaCobroBanco'] = pd.to_datetime(detalle['fechaCobroBanco'], errors='coerce')
detalle['montoCobrado'] = pd.to_numeric(detalle['montoCobrado'], errors='coerce')
detalle['montoCobrar'] = pd.to_numeric(detalle['montoCobrar'], errors='coerce')
detalle['montoExigible'] = pd.to_numeric(detalle.get('montoExigible', 0), errors='coerce')

# -----------------------------
# PARTE 2: CLASIFICACIÓN
# -----------------------------
hoy = pd.Timestamp.now()

def hubo_exito_reciente(df):
    recientes = df[df['fechaCobroBanco'] > hoy - pd.Timedelta(days=365)]
    return (recientes['montoCobrado'] > 0).any()

resumen = detalle.groupby('idCredito').apply(lambda g: pd.Series({
    'intentos_totales': g.shape[0],
    'intentos_exitosos': (g['montoCobrado'] > 0).sum(),
    'intentos_bloqueados': g['es_bloqueo'].sum(),
    'ultimo_exito': g[g['montoCobrado'] > 0]['fechaCobroBanco'].max(),
    'primer_intento': g['fechaCobroBanco'].min(),
    'monto_total': g['montoCobrar'].sum(),
    'exito_reciente': hubo_exito_reciente(g),
    'num_estrategias_diferentes': g['estrategia_simplificada'].nunique()
})).reset_index()

resumen['antiguedad_anios'] = (hoy - resumen['primer_intento']).dt.days / 365
resumen['sin_exito_desde'] = (hoy - resumen['ultimo_exito']).dt.days / 365
resumen['clasificacion'] = 'sin_clasificar'
resumen.loc[resumen['intentos_bloqueados'] > 0, 'clasificacion'] = 'interjudicial'
resumen.loc[(resumen['clasificacion'] == 'sin_clasificar') & (
    (resumen['intentos_totales'] >= 10) |
    (resumen['antiguedad_anios'] >= 4) |
    (resumen['monto_total'] < 50)), 'clasificacion'] = 'dar_por_perdido'
resumen.loc[(resumen['clasificacion'] == 'sin_clasificar') & (resumen['exito_reciente']),
            'clasificacion'] = 'mantener_estrategia'
resumen.loc[(resumen['clasificacion'] == 'sin_clasificar') &
            (resumen['num_estrategias_diferentes'] == 1), 'clasificacion'] = 'probar_otra_estrategia'
resumen.loc[resumen['clasificacion'] == 'sin_clasificar', 'clasificacion'] = 'intentar_por_bbva'

# -----------------------------
# PARTE 3: ENTRENAMIENTO
# -----------------------------
X1 = resumen[[
    'intentos_totales', 'intentos_exitosos', 'intentos_bloqueados',
    'monto_total', 'antiguedad_anios', 'sin_exito_desde', 'num_estrategias_diferentes'
]].fillna(0)

y1 = resumen['clasificacion']
label_encoder1 = LabelEncoder()
y1_encoded = label_encoder1.fit_transform(y1)

scaler1 = StandardScaler()
X1_scaled = scaler1.fit_transform(X1)

X1_train, X1_test, y1_train, y1_test = train_test_split(X1_scaled, y1_encoded, test_size=0.2, random_state=42)
y1_train_cat = to_categorical(y1_train)
y1_test_cat = to_categorical(y1_test)

model1 = Sequential([
    Dense(64, activation='relu', input_shape=(X1_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(y1_train_cat.shape[1], activation='softmax')
])
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model1.fit(X1_train, y1_train_cat, epochs=20, batch_size=512, validation_split=0.2)

# -----------------------------
# PARTE 4: GUARDADO FINAL
# -----------------------------
model1.save("/mnt/data/modelo_clasificacion_estrategia.h5")
joblib.dump(scaler1, "/mnt/data/scaler_clasificacion.pkl")
joblib.dump(label_encoder1, "/mnt/data/label_encoder_clasificacion.pkl")


FileNotFoundError: [Errno 2] No such file or directory: 'Tabla_con_Observaciones.xlsx'